In [ ]:
import cv2, numpy as np
from json import dumps
from flask import Flask
from mediapipe import solutions

In [ ]:
app = Flask(__name__)
cap = cv2.VideoCapture(0)
WIDTH, HEIGHT = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
mp_face_mesh = solutions.face_mesh
face = mp_face_mesh.FaceMesh(min_tracking_confidence = 0.6, min_detection_confidence = 0.6)
draw = solutions.drawing_utils

In [ ]:
with mp_face_mesh.FaceMesh() as face_mesh:
    while True:
        ok, frame = cap.read()
        if not ok:
            continue
        op = face.process(frame)
        if op.multi_face_landmarks:
            face_landmarks = op.multi_face_landmarks[0].landmark
            coord_centro = np.array(solutions.drawing_utils._normalized_to_pixel_coordinates(
                face_landmarks[1].x, face_landmarks[1].y, WIDTH, HEIGHT
            ))
            cv2.imshow("Video", cv2.putText(cv2.rectangle(frame, coord_centro-np.array([5, 5]), coord_centro+np.array([5, 5]), (0, 255, 0)), f"Centro: {coord_centro[0], coord_centro[1]}", (0, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0)))
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

In [ ]:
@app.route("/coord")
def get_face_center():
    ok, frame = cap.read()
    if not ok:
        return dumps([0, 0])
    op = face.process(frame)
    if op.multi_face_landmarks:
        face_landmarks = op.multi_face_landmarks[0].landmark
        return dumps(solutions.drawing_utils._normalized_to_pixel_coordinates(
            face_landmarks[1].x, face_landmarks[1].y, WIDTH, HEIGHT
        ))

In [ ]:
app.run(port=8080)